# Assignment 1

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_core.tools import tool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

movie_infos = []
@tool
def movie_infos_search(movie_name: str):
    """
    Input the movie name that the user is looking for and extract movies from IMDb search results and return their infos.
    The infos include the movie title, plot, cast and characters.
    """
    
    chrome_options = Options()
    driver = webdriver.Chrome(options = chrome_options)
    driver.minimize_window()
    # driver.maximize_window()

    try:
        url = "https://www.imdb.com/?ref_=nv_home"
        driver.get(url)

        input = driver.find_element(By.ID, "suggestion-search")
        input.send_keys(movie_name)

        button = driver.find_element(By.ID, "suggestion-search-button")
        button.click()
        time.sleep(0.1)

        li_elements = driver.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item')
        for i in range(5):
            metadatas_element = li_elements[i].find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item__li')
            metadatas = []
            for metadata_element in metadatas_element:
                metadatas.append(metadata_element.text)

            if 'TV Series' in metadatas or 'Video' in metadatas:
                # print('Skipping')
                continue

            title_element = li_elements[i].find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
            title = title_element.text
            link = title_element.get_attribute("href")
            movie_infos.append({"title": title, "link": link, "metadatas": metadatas})

        # for movie_info in movie_infos:
        #     print(movie_info)

        for i in range(len(movie_infos)):
            # print(movie_infos[i]["title"])
            link = movie_infos[i]["link"]
            driver.get(link)

            section = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section')
            p = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p')
            plot_text = p.text
            movie_infos[i]["plot"] = plot_text
            if "Plot under wraps" in plot_text:
                movie_infos[i]["casts_and_characters"] = []
                continue

            try:
                section = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section')
                section = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]')
                names = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]')
                names_list = names.text.splitlines()
            except Exception as e:
                names_list = []

            casts_and_characters = []
            for j in range(0, len(names_list), 2):
                names_list[j] = names_list[j].replace("…", "").replace("(voice)", "")
                names_list[j + 1] = names_list[j + 1].replace("…", "").replace("(voice)", "")
                casts_and_characters.append({"cast": names_list[j], "character": names_list[j + 1]})
            movie_infos[i]["casts_and_characters"] = casts_and_characters

        # for movie_info in movie_infos:
        #     print(movie_info)

        response = ""
        for movie_info in movie_infos:
            response += f"""
Movie Title: {movie_info["title"]}
Link: {movie_info["link"]}
Plot: {movie_info["plot"]}
Casts and Characters:
"""
            for cast_and_character in movie_info["casts_and_characters"]:
                response += f"  {cast_and_character["cast"]} playing {cast_and_character["character"]}\n"

        driver.quit()
        return response
    except Exception as e:
        # movie_infos = []
        driver.quit()
        return str(e)

@tool
def movie_metadata_search(url: str):
    """
    Input the IMDb movie page URL and extract the movie title, plot, casts, characters, directors, writers, rating, popularity.
    """
    response = ""
    for movie_info in movie_infos:
        if movie_info["link"] == url:
            chrome_options = Options()
            driver = webdriver.Chrome(options = chrome_options)
            driver.maximize_window()
            
            driver.get(url)

            rating = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]')
            movie_info["rating"] = rating.text
            # print("Rating: " + rating.text)

            popularity = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]')
            movie_info["popularity"] = popularity.text
            # print("Popularity: " + popularity.text)

            driver.minimize_window()

            # Director
            ul_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div/ul')
            li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
            directors = []
            for li in li_elements:
                directors.append(li.text)
                # print(li.text)
            movie_info["directors"] = directors

            # Writer
            ul_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div/ul')
            li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
            writers = []
            for li in li_elements:
                writers.append(li.text)
                # print(li.text)
            movie_info["writers"] = writers

            box_office_div = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="title-boxoffice-section"]')

            box_office_text = box_office_div.text
            lines = box_office_text.splitlines()
            lines = lines[:-1]
            box_office = []
            for i in range(0, len(lines), 2):
                box_office.append(lines[i] + ": " + lines[i + 1])
            # print(box_office)
            movie_info["box_office"] = box_office
            
            response += f"""
Movie Title: {movie_info["title"]}
Link: {movie_info["link"]}
Plot: {movie_info["plot"]}
Casts and Characters:
"""
            for cast_and_character in movie_info["casts_and_characters"]:
                response += f"  {cast_and_character["cast"]} playing {cast_and_character["character"]}\n"
            response += "Directors: "
            for director in movie_info["directors"]:
                response += f"{director}, "
            response += "\nWriters: "
            for writer in movie_info["writers"]:
                response += f"{writer}, "
            response += "\nBox Office:\n"
            for box in movie_info["box_office"]:
                response += f"  {box}\n"
            response += f"""Rating: {movie_info["rating"]}
Popularity: {movie_info["popularity"]}
"""
            
            driver.quit()
            return response

In [ ]:
# movie_infos_text = movie_infos_search.invoke("The Matrix")
# print(movie_infos_text)

In [ ]:
# movie_metadata_text = movie_metadata_search.invoke("https://www.imdb.com/title/tt0133093/?ref_=fn_all_ttl_1")
# print(movie_metadata_text)

In [ ]:
from langgraph.prebuilt import create_react_agent

prompt = """
    Based on the plan steps provided by planner, please select the appropriate tool to use and provide the response of the plan step.
"""

tools = [movie_infos_search, movie_metadata_search]
executor = create_react_agent(model, tools, prompt=prompt)

In [ ]:
# from langchain_core.messages import HumanMessage

# # Terminator 2: I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The mvoie is the second installment in a series.
# # The Avengers: I am looking for a movie that starring Robert Downey Jr., Chris Evans, and Scarlett Johansson.
# # The Matrix: I am looking for a movie starring Keanu Reeves and the movie is about virtual reality.
# movie_infos = []
# query = "I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The mvoie is the second installment in a series."

# config = {"configurable": {"thread_id": "abc123"}}
# for step in executor.stream(
#     {"messages": [HumanMessage(content=query)]},
#     config,
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

# Assignment 2

In [ ]:
import operator
from typing import Annotated, List, Tuple
from typing_extensions import TypedDict
from langgraph.graph import MessagesState



class PlanExecute(MessagesState):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan.  
This plan should involve individual tasks, that if executed correctly will yield the correct answer.  
Do not add any superfluous steps.  
The result of the final step should be the final answer.  
Make sure that each step has all the information needed - do not skip steps.

You can reference the following plan as an example:
    1. Generate a probable movie name based on user input  
    Goal: Extract or infer the most likely movie title the user is referring to from their natural language query.  
    Method: Apply semantic understanding, contextual reasoning, named entity recognition (NER), or prompted inference to transform the raw query into a clear movie name.

    2. Search IMDb and determine the correct movie based on search results  
    Goal: Use the inferred movie name from the previous step to perform an IMDb search and select the most semantically relevant result.  
    Method:  
    Use the movie_search(movie_name) tool to retrieve multiple search results  
    Compare result titles, release years, actors, and other metadata to identify the best match

    3. Retrieve detailed information about the selected movie from IMDb  
    Goal: Use the selected IMDb link to retrieve comprehensive movie metadata such as plot, cast, characters, directors, ratings, etc.  
    Method: Call the movie_metadata_search(imdb_url) tool

At each step, clearly output structured results such as:  
- Step 1 → movie_name: <string>  
- Step 2 → selected_movie_url: <IMDb URL>  
- Step 3 → metadata: <JSON object with fields such as title, plot, casts, etc.>      

If the provided movie is not what the user is looking for, start over and create a new plan by generating alternative possible movie names based on the user’s previous input and searching again.

Examples of mismatch conditions (triggering replan):  
- The plot returned is semantically inconsistent with the original user query.  
- None of the cast members match the ones mentioned by the user.  
- The title or year obviously contradicts contextual hints.
""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4o-mini", temperature=0
).with_structured_output(Plan)

In [ ]:
response = planner.invoke(
    {
        "messages": [
            ("user", "I am looking for a movie starring Keanu Reeves and the movie is about virtual reality.")
        ]
    }
)
for step in response.steps:
    print(step)

In [ ]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. 
If no more steps are needed and you can return to the user, then respond with that. 
If you need more information from the user to complete the task, then respond with that.
Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.

Notice:
Do not respond to the user with the plan. You should only return the final conclusion to the user.
"""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4o", temperature=0
).with_structured_output(Act)

In [ ]:
from typing import Literal
from langgraph.graph import END


async def execute_step(state: PlanExecute):
    user_input = state["input"]
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""User Input: {user_input}\n\n 
For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await executor.ainvoke(
        {"messages": [("user", task_formatted)]},
        config={"configurable": {"thread_id": "executor123"}}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }


async def plan_step(state: PlanExecute):
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    
    if "response" in state and state["response"]:
        return {"plan": plan.steps, "response": ""}
    
    return {"plan": plan.steps}


async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "agent"

In [ ]:
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(PlanExecute)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("agent", execute_step)

# Add a replan node
workflow.add_node("replanner", replan_step)

workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "agent")

# From agent, we replan
workflow.add_edge("agent", "replanner")

workflow.add_conditional_edges(
    "replanner",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["agent", END],
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
memeory = MemorySaver()
app = workflow.compile(checkpointer=memeory)

In [ ]:
# from IPython.display import Image, display

# display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "I am looking for a movie starring Keanu Reeves and the movie is about virtual reality."}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)
            if "response" in v:
                print(v["response"])

In [ ]:
config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "Please provide more details of the movie, such as characters, directors, and ratings."}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)
            if "response" in v:
                print(v["response"])

In [ ]:
# # I am looking for a movie starring Keanu Reeves and the movie is about virtual reality.
# # Please provide more details of the movie, such as characters, directors, and ratings.

# while True:
#     # Get user input
#     user_input = input("Please enter your query: ")
#     if user_input.lower() == "exit":
#         break
#     config = {"recursion_limit": 50, "thread_id": "workflow123"}
#     inputs = {"input": user_input}
#     async for event in app.astream(inputs, config=config):
#         for k, v in event.items():
#             if k != "__end__":
#                 print(v)
#                 if "response" in v:
#                     print(v["response"])

# Assignment 3

## API Keys & LLM Models

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

model = init_chat_model("gpt-4.1-mini", model_provider="openai")

## Tools

In [2]:
from langchain_core.tools import tool
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

@tool
def movie_name_generate(user_input: str):
    """
    Input the user input and generate a movie name that the user is looking for.
    """
    embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

    vector_store = Chroma(
        collection_name="movie_infos",
        embedding_function=embeddings,
        persist_directory="./movie_infos_db",
    )

    results = vector_store.similarity_search(user_input, k=1)

    print(f"Successfully retrieved the movie name {results[0].metadata['title']} from the database.")
    return results[0].metadata["title"]

movie_infos = []
@tool
def movie_infos_search(movie_name: str):
    """
    Input the movie name that the user is looking for and extract movies from IMDb search results and return their infos.
    The infos include the movie title, plot, cast and characters.
    """
    
    chrome_options = Options()
    driver = webdriver.Chrome(options = chrome_options)
    driver.minimize_window()
    # driver.maximize_window()

    try:
        url = "https://www.imdb.com/?ref_=nv_home"
        driver.get(url)

        input = driver.find_element(By.ID, "suggestion-search")
        input.send_keys(movie_name)

        button = driver.find_element(By.ID, "suggestion-search-button")
        button.click()
        time.sleep(0.1)

        movie_infos_temp = []
        li_elements = driver.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item')
        for i in range(5):
            metadatas_element = li_elements[i].find_elements(By.CLASS_NAME, 'ipc-metadata-list-summary-item__li')
            metadatas = []
            for metadata_element in metadatas_element:
                metadatas.append(metadata_element.text)

            if 'TV Series' in metadatas or 'Video' in metadatas:
                # print('Skipping')
                continue

            title_element = li_elements[i].find_element(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
            title = title_element.text
            link = title_element.get_attribute("href")
            movie_infos_temp.append({"title": title, "link": link, "metadatas": metadatas})
        movie_infos = movie_infos_temp

        # for movie_info in movie_infos:
        #     print(movie_info)

        for i in range(len(movie_infos)):
            # print(movie_infos[i]["title"])
            link = movie_infos[i]["link"]
            driver.get(link)

            section = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section')
            p = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p')
            plot_text = p.text
            movie_infos[i]["plot"] = plot_text
            if "Plot under wraps" in plot_text:
                movie_infos[i]["casts_and_characters"] = []
                continue

            try:
                section = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section')
                section = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]')
                names = section.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]')
                names_list = names.text.splitlines()
            except Exception as e:
                names_list = []

            casts_and_characters = []
            for j in range(0, len(names_list), 2):
                names_list[j] = names_list[j].replace("…", "").replace("(voice)", "")
                names_list[j + 1] = names_list[j + 1].replace("…", "").replace("(voice)", "")
                casts_and_characters.append({"cast": names_list[j], "character": names_list[j + 1]})
            movie_infos[i]["casts_and_characters"] = casts_and_characters

        # for movie_info in movie_infos:
        #     print(movie_info)

        response = ""
        for movie_info in movie_infos:
            response += f"""
Movie Title: {movie_info["title"]}
Link: {movie_info["link"]}
Plot: {movie_info["plot"]}
Casts and Characters:
"""
            for cast_and_character in movie_info["casts_and_characters"]:
                response += f"  {cast_and_character["cast"]} playing {cast_and_character["character"]}\n"

        driver.quit()
        return response
    except Exception as e:
        # movie_infos = []
        driver.quit()
        return str(e)

@tool
def movie_metadata_search(url: str):
    """
    Input the IMDb movie page URL and extract the movie title, plot, casts, characters, directors, writers, rating, popularity.
    """
    response = ""
    for movie_info in movie_infos:
        if movie_info["link"] == url:
            chrome_options = Options()
            driver = webdriver.Chrome(options = chrome_options)
            driver.maximize_window()
            
            driver.get(url)

            rating = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]')
            movie_info["rating"] = rating.text
            # print("Rating: " + rating.text)

            popularity = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]')
            movie_info["popularity"] = popularity.text
            # print("Popularity: " + popularity.text)

            driver.minimize_window()

            # Director
            ul_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[1]/div/ul')
            li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
            directors = []
            for li in li_elements:
                directors.append(li.text)
                # print(li.text)
            movie_info["directors"] = directors

            # Writer
            ul_element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[4]/ul/li[2]/div/ul')
            li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
            writers = []
            for li in li_elements:
                writers.append(li.text)
                # print(li.text)
            movie_info["writers"] = writers

            box_office_div = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="title-boxoffice-section"]')

            box_office_text = box_office_div.text
            lines = box_office_text.splitlines()
            lines = lines[:-1]
            box_office = []
            for i in range(0, len(lines), 2):
                box_office.append(lines[i] + ": " + lines[i + 1])
            # print(box_office)
            movie_info["box_office"] = box_office
            
            response += f"""
Movie Title: {movie_info["title"]}
Link: {movie_info["link"]}
Plot: {movie_info["plot"]}
Casts and Characters:
"""
            for cast_and_character in movie_info["casts_and_characters"]:
                response += f"  {cast_and_character["cast"]} playing {cast_and_character["character"]}\n"
            response += "Directors: "
            for director in movie_info["directors"]:
                response += f"{director}, "
            response += "\nWriters: "
            for writer in movie_info["writers"]:
                response += f"{writer}, "
            response += "\nBox Office:\n"
            for box in movie_info["box_office"]:
                response += f"  {box}\n"
            response += f"""Rating: {movie_info["rating"]}
Popularity: {movie_info["popularity"]}
"""
            
            driver.quit()
            return response

In [3]:
result = movie_name_generate.invoke({"user_input": "I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The mvoie is the second installment in a series."})
print(result)

C:\Users\George\AppData\Local\Temp\ipykernel_25040\873716119.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
C:\Users\George\AppData\Local\Temp\ipykernel_25040\873716119.py:16: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


Successfully retrieved the movie name 機械公敵 from the database.
機械公敵


## Agents

In [ ]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

class Plan(BaseModel):
    """Plan to follow in future"""

    steps: List[str] = Field(
        description="different steps to follow, should be in sorted order"
    )

planner_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """For the given objective, come up with a simple step by step plan.  
This plan should involve individual tasks, that if executed correctly will yield the correct answer.  
Do not add any superfluous steps.  
The result of the final step should be the final answer.  
Make sure that each step has all the information needed - do not skip steps.

You can reference the following plan as an example:
    1. Generate a probable movie name based on user query.
    Goal: Extract or infer the most likely movie title the user is referring to from their natural language query.  
    Method: Use the movie_name_generate(user_input) tool to retrieve the most relevant movie name based on the user input.

    2. Search IMDb and determine the correct movie based on search results  
    Goal: Use the inferred movie name from the previous step to perform an IMDb search and select the most semantically relevant result.  
    Method:  
    Use the movie_search(movie_name) tool to retrieve multiple search results  
    Compare result titles, release years, actors, and other metadata to identify the best match

    3. Retrieve detailed information about the selected movie from IMDb  
    Goal: Use the selected IMDb link to retrieve comprehensive movie metadata such as plot, cast, characters, directors, ratings, etc.  
    Method: Call the movie_metadata_search(imdb_url) tool

At each step, clearly output structured results such as:  
    - Step 1 → movie_name: <string>  
    - Step 2 → selected_movie_url: <IMDb URL>  
    - Step 3 → metadata: <JSON object with fields such as title, plot, casts, etc.>      

If the provided movie is not what the user is looking for, start over and create a new plan by generating alternative possible movie names based on the user’s previous input and searching again.

Examples of mismatch conditions (triggering replan):  
    - The plot returned is semantically inconsistent with the original user input.  
    - None of the cast members match the ones mentioned by the user.  
    - The title or year obviously contradicts contextual hints.
""",
        ),
        ("placeholder", "{messages}"),
    ]
)
planner = planner_prompt | ChatOpenAI(
    model="gpt-4.1-mini", temperature=0
).with_structured_output(Plan)

In [ ]:
response = planner.invoke(
    {
        "messages": [
            ("user", "I am looking for a movie starring Keanu Reeves and the movie is about virtual reality.")
        ]
    }
)
for step in response.steps:
    print(step)

In [ ]:
from langgraph.prebuilt import create_react_agent

prompt = """
    Based on the plan steps provided by planner, please select the appropriate tool to use and provide the response of the plan step.
"""

tools = [movie_name_generate, movie_infos_search, movie_metadata_search]
executor = create_react_agent(model, tools, prompt=prompt)

In [ ]:
from typing import Union


class Response(BaseModel):
    """Response to user."""

    response: str


class Act(BaseModel):
    """Action to perform."""

    action: Union[Response, Plan] = Field(
        description="Action to perform. If you want to respond to user, use Response. "
        "If you need to further use tools to get the answer, use Plan."
    )


replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. 
If no more steps are needed and you can return to the user, then respond with that. 
If you need more information from the user to complete the task, then respond with that.
Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan.

Notice:
    Every step in the plan should be executed in order.
    Do not respond to the user with the plan. You should only return the final conclusion to the user.
"""
)


replanner = replanner_prompt | ChatOpenAI(
    model="gpt-4.1-mini", temperature=0
).with_structured_output(Act)

## Graph

In [ ]:
import operator
from typing import Annotated, List, Tuple
from langgraph.graph import MessagesState

class PlanExecute(MessagesState):
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

In [ ]:
import shutil
from typing import List
from langchain_core.documents import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma

shutil.rmtree("./plan_documents_db", ignore_errors=True)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

plan_docs_vector_store = Chroma(
    collection_name="plan_documents",
    embedding_function=embeddings,
    persist_directory="./plan_documents_db",
)

plan_docs = []

def save_plan_docs(user_input: str, plan: List[str]):
    plan_str = "\n".join(plan)

    doc = Document(
        page_content=(
            user_input
        ),
        metadata={
            "user_input": user_input,
            "plan": plan_str,
        }
    )

    plan_docs.append(doc)
    plan_docs_vector_store.add_documents([doc])

    print("Successfully saved the plan documents to the vector store.")

In [ ]:
from langgraph.graph import END

def retrieve_plan_doc(state: PlanExecute):
    user_input = state["input"]
    results = plan_docs_vector_store.similarity_search_with_score(user_input, k=1)
    
    if not results:
        print("No plan found in the vector store.")
        return {"plan": None}
    
    score = results[0][1]
    if score > 0.5:
        print("No plan found in the vector store.")
        plan = None
    else:
        print("Successfully retrieved the plan from the vector store. Socre:", score)
        plan = results[0][0].metadata["plan"].split("\n")
    
    return {"plan": plan}

def should_plan(state: PlanExecute):
    if not state["plan"]:
        return "planner"
    else:
        return "executor"
    
async def plan_step(state: PlanExecute):
    # * 傳入上次執行結果
    if "response" in state and state["response"]:
        plan = await planner.ainvoke({"messages": [
            ("user", "Last Respone: " + state["response"]),
            ("user", f"User Input: {state["input"]}"),
        ]})

        save_plan_docs(state["input"], plan.steps)

        return {"plan": plan.steps, "response": ""}
    
    plan = await planner.ainvoke({"messages": [
        ("user", f"User Input: {state["input"]}"),
    ]})

    save_plan_docs(state["input"], plan.steps)

    return {"plan": plan.steps}

async def execute_step(state: PlanExecute):
    user_input = state["input"]
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"""User Input: {user_input}\n\n 
For the following plan:
{plan_str}\n\nYou are tasked with executing step {1}, {task}."""
    agent_response = await executor.ainvoke(
        {"messages": [("user", task_formatted)]},
        config={"configurable": {"thread_id": "executor123"}}
    )
    return {
        "past_steps": [(task, agent_response["messages"][-1].content)],
    }

async def replan_step(state: PlanExecute):
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}


def should_end(state: PlanExecute):
    if "response" in state and state["response"]:
        return END
    else:
        return "executor"

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

workflow = StateGraph(PlanExecute)

# Add the retrive node
workflow.add_node("retriever", retrieve_plan_doc)

# Add the plan node
workflow.add_node("planner", plan_step)

# Add the execution step
workflow.add_node("executor", execute_step)

# Add a replan node
workflow.add_node("replanner", replan_step)

workflow.add_edge(START, "retriever")

workflow.add_conditional_edges(
    "retriever",
    should_plan,
    ["planner", "executor"],
)

# workflow.add_edge(START, "planner")

# From plan we go to agent
workflow.add_edge("planner", "executor")

# From agent, we replan
workflow.add_edge("executor", "replanner")

workflow.add_conditional_edges(
    "replanner",
    # Next, we pass in the function that will determine which node is called next.
    should_end,
    ["executor", END],
)

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

## Test(lab)

In [ ]:
#  I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The movie is the second installment in a series.
#  I am looking for a movie starring Keanu Reeves and the movie is about virtual reality.

config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The movie is the second installment in a series."}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(k, "\n", v)
            if "response" in v:
                print()
                print("Final Response:", v["response"])
            print()

In [ ]:
print(len(plan_docs))
print(plan_docs)

In [ ]:
results = plan_docs_vector_store.similarity_search_with_score("Keanu Reeves", k=5)

for result in results:
    print(result[0].page_content, result[1])
    plan_list = result[0].metadata["plan"].split("\n")
    print(plan_list)
    print()

In [ ]:
config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The movie is the second installment in a series."}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(k, "\n", v)
            if "response" in v:
                print()
                print("Final Response:", v["response"])
            print()

In [ ]:
print(len(plan_docs))
print(plan_docs)

In [ ]:
results = plan_docs_vector_store.similarity_search_with_score("I am looking for a movie about virtual reality", k=5)
for result in results:
    print(result[0].page_content, result[1])
    plan_list = result[0].metadata["plan"].split("\n")
    print(plan_list)
    print()

In [ ]:
config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "I am looking for a movie that starring Robert Downey Jr., Chris Evans, and Scarlett Johansson."}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(k, "\n", v)
            if "response" in v:
                print()
                print("Final Response:", v["response"])
            print()

In [ ]:
print(len(plan_docs))
print(plan_docs)

In [ ]:
results = plan_docs_vector_store.similarity_search_with_score("I am looking for a movie about virtual reality", k=5)
for result in results:
    print(result[0].page_content, result[1])
    plan_list = result[0].metadata["plan"].split("\n")
    print(plan_list)
    print()

# Test(FilmSeeker_v3)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

from graph import graph

config = {"recursion_limit": 50, "thread_id": "workflow123"}
inputs = {"input": "I am looking for a movie that a robot pretending to be a human and trying to assassinate the main character. The movie is the second installment in a series."}
async for event in graph.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(k, "\n", v)
            if "response" in v:
                print()
                print("Final Response:", v["response"])
            print()